In [ ]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir
from os.path import join
from sklearn import metrics
import statsmodels
from statsmodels.graphics.tsaplots import plot_acf

# Data uploading

The data was generated form the AR(1) model of the following form $y_t=N(0,1)-0.6y_{t-1}$

Size of the data set: t=1000

In [ ]:
d1=pd.read_csv("AR.csv")
data=np.array(d1.x)

In [ ]:
plot_acf(data, lags=31)
plt.show()

# Data preparation

Arguments: 

-time_wind: length of the sequence of the historical data;

-output: how many steps ahead prediction is going to be performed;

-nb_var: number of variables of the data set;

In [ ]:
def data_preparation(data,time_wind=2,output=1,nb_var=1,batch_size=1):
    data_train=data[:901]
    x_train=[]
    for i in range((len(data_train)-1)-(time_wind-1)):
        x_train.append(data_train[i:(i+time_wind)])
    x_train=np.array(x_train)
    x_batches=(x_train[:(len(x_train)-(len(x_train)%batch_size))]).reshape(-1,time_wind,nb_var)
    y_train=data_train[time_wind:len(data_train)]
    y_batches=(y_train[:(len(y_train)-(len(y_train)%batch_size))]).reshape(-1,output)
    data_test=data[902:]
    x_t=[]
    for i in range((len(data_test)-1)-(time_wind-1)):
        x_t.append(data_test[i:(i+time_wind)])
    x_t=np.array(x_t)
    x_test=(x_t[:(len(x_t)-(len(x_t)%batch_size))]).reshape(-1,time_wind,nb_var)
    y_t=data_test[time_wind:len(data_test)]
    y_test=(y_t[:(len(y_t)-(len(y_t)%batch_size))]).reshape(-1,output)
    return x_batches, y_batches, x_test, y_test

# TensorFlow graph

Arguments:

-epochs: how many times the network is going to be trained;

-time_wind: the length of the historical sequence;

-hidden: number of hidden layers;

-nb_var: amount of input variables;

-output: how many steps ahead prediction is going to be performed;

-lr: learning rate.

Outputs:

-mse_test: the mean squared error for the final prediction vector;

-y_pred: the vector of predictions.

In [ ]:
def run_graph(epochs,time_wind,hidden=10,nb_var=1,output=1,lr=0.01,r_s=1):

    from tensorflow.contrib import rnn
    tf.reset_default_graph()
    X=tf.placeholder(tf.float32,[None,time_wind,nb_var])
    Y=tf.placeholder(tf.float32,[None,output])
 
    tf.set_random_seed(r_s)
    x=tf.unstack(X,time_wind,1)
    
    basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden)
    rnn_output, states=rnn.static_rnn(basic_cell,x,dtype=tf.float32)

    stacked_rnn_output=tf.reshape(rnn_output[-1],[-1,hidden])
    stacked_outputs=tf.layers.dense(stacked_rnn_output,output)
    loss=tf.reduce_mean(tf.squared_difference(stacked_outputs, Y))
    optimizer=tf.train.AdamOptimizer(learning_rate=lr)
    training_op=optimizer.minimize(loss)
    init=tf.global_variables_initializer()
    with tf.Session() as sess:
        init.run()
        for ep in range(epochs):
            sess.run(training_op,feed_dict={X: x_batches, Y: y_batches})
            if ep % 1000 == 0:
                mse=loss.eval(feed_dict={X: x_batches, Y: y_batches})
                print(ep,"\tMSE:",mse)
        y_pred=sess.run(stacked_outputs,feed_dict={X:x_test})
        mse_test=loss.eval(feed_dict={X:x_test,Y:y_test})
        print(mse)
        print(mse_test)
    sess.close() 
    return mse_test, y_pred

# Visualisation

In [ ]:
def visualisation():
    plt.plot(y_test,label='Actual')
    plt.plot(y_pred,label='Forecast')
    plt.legend(loc="upper left")
    return plt.show

# Expariments
## Naive models
### Average naive model

In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(data,1)
print(metrics.mean_squared_error(x_batches[:,0],np.full(len(x_batches),np.mean(x_batches))))

### Constant naive model

In [ ]:
x_t1=y_test[:96]
x_t=y_test[1:]
print(metrics.mean_squared_error(x_t,x_t1))

# Time_wind

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(1,9,1):
    mse=[]
    for i in range (10):
        x_batches, y_batches, x_test, y_test=data_preparation(data,j)
        mse_test,y_pred=run_graph(1000,j,hidden=1,lr=0.1,r_s=i)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with time_wind={} equals{}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with time_wind={} equals{}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

Summary: it is no need to widen the time window. What is according to our expectations (as the data was generated form the process AR(1))

# Hidden layers

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(1,11,1):
    mse=[]
    for i in range (10):
        x_batches, y_batches, x_test, y_test=data_preparation(data,1)
        mse_test,y_pred=run_graph(1000,1,hidden=j,lr=0.1,r_s=i)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with hidden={} equals{}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with hidden={} equals{}".format(j, np.var(mse)))
visualisation()
    

In [ ]:
plt.plot(mse_mean)

Summary: The best accuracy is achieved with the hidden=10. MSE=1.004931. As hidden=10 was the last value that was tested during the experiment, it makes sense to perform one more, in order to check the next range of sizes of the hidden state.

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(10,31,1):
    mse=[]
    for i in range (10):
        x_batches, y_batches, x_test, y_test=data_preparation(data,1)
        mse_test,y_pred=run_graph(1000,1,hidden=j,lr=0.1,r_s=i)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with hidden={} equals{}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with hidden={} equals{}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

Summary: The highest accuracy is achieved with size of the hidden state equal to hidden=18. MSE=1.0048877

# Optimisation of the training time
## Learning rate: 0.1

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(1000,11000,1000):
    mse=[]
    for i in range (10):
        x_batches, y_batches, x_test, y_test=data_preparation(data,1)
        mse_test,y_pred=run_graph(j,1,hidden=18,lr=0.1,r_s=i)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with epochs={} equals {}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with epochs={} equals {}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

Summary:  With the learning rate equal to 0.01, the highest accuracy is achieved with the number of epochs equal to 5000. The MSE=0.99077922

## Learning rate: 0.01

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(3000,11000,1000):
    mse=[]
    for i in range (10):
        x_batches, y_batches, x_test, y_test=data_preparation(data,1)
        mse_test,y_pred=run_graph(j,1,hidden=18,lr=0.01,r_s=i)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with epochs={} equals {}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with epochs={} equals {}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

Summary: With the lr=0.01, the lowest MSE is echieved with number of training epochs equal to 8000. MSE= 0.98785669

## Learning rate: 0.001

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(6000,14000,1000):
    mse=[]
    for i in range (10):
        x_batches, y_batches, x_test, y_test=data_preparation(data,1)
        mse_test,y_pred=run_graph(j,1,hidden=18,lr=0.001,r_s=i)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with epochs={} equals {}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with epochs={} equals {}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

Summary: The highest accuracy is achived with the number of training epochs 14000. MSE=0.99555.  However, looking on the  dynamic of the changes of the accuracy, we get the conclusion that experiment should be continuing on the following range of the training time.

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(14000,22000,1000):
    mse=[]
    for i in range (10):
        x_batches, y_batches, x_test, y_test=data_preparation(data,1)
        mse_test,y_pred=run_graph(j,1,hidden=18,lr=0.001,r_s=i)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with epochs={} equals {}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with epochs={} equals {}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

Summary: the highest accuracy is achieved with number of epochs equal to 19000. MSE=0.98768318